<img src="https://app.powerbi.com/13.0.10537.111/images/PowerBI96x96.png?raw=true" width="10%"> ➕
<img src="https://github.com/Microsoft/azuredatastudio/blob/master/samples/notebookSamples/Graphics/AzureDataStudioLogo.png?raw=true" width="10%"> = ❤

# Copying Workspaces with PowerShell


If you don't already have the Power BI PowerShell module installed, you can do that now by running the `Install-Module` command below.

In [ ]:
Install-Module -Name MicrosoftPowerBIMgmt -Scope CurrentUser

Before you can get any of the Power BI cmdlets to work, you first need to login to your tenant.  

Once you login, your credentials will be used for all subsequent Power BI commands in this Notebook.  After running the command below, you should not have to run `Login-PowerBI` again until you either: A) Close & re-open this Notebook.  B) run the `Logout-PowerBI` command.

In [4]:
Login-PowerBI

Here's a list of all the PowerShell cmdlets available for Power BI.

In [1]:
Get-Command -Module MicrosoftPowerBI* -CommandType Cmdlet


CommandType     Name                                               Version    Source                                   
-----------     ----                                               -------    ------                                   
Cmdlet          Add-PowerBIDataset                                 1.0.802    MicrosoftPowerBIMgmt.Data                
Cmdlet          Add-PowerBIEncryptionKey                           1.0.802    MicrosoftPowerBIMgmt.Admin               
Cmdlet          Add-PowerBIRow                                     1.0.802    MicrosoftPowerBIMgmt.Data                
Cmdlet          Add-PowerBIWorkspaceUser                           1.0.802    MicrosoftPowerBIMgmt.Workspaces          
Cmdlet          Connect-PowerBIServiceAccount                      1.0.802    MicrosoftPowerBIMgmt.Profile             
Cmdlet          Copy-PowerBIReport                                 1.0.802    MicrosoftPowerBIMgmt.Reports             
Cmdlet          Copy-PowerBITile       

Here are the cmdlets available for working with Workspaces

In [2]:
Get-Command -Module MicrosoftPowerBI* -Noun *Workspace*


CommandType     Name                                               Version    Source                                   
-----------     ----                                               -------    ------                                   
Cmdlet          Add-PowerBIWorkspaceUser                           1.0.802    MicrosoftPowerBIMgmt.Workspaces          
Cmdlet          Get-PowerBIWorkspace                               1.0.802    MicrosoftPowerBIMgmt.Workspaces          
Cmdlet          Get-PowerBIWorkspaceEncryptionStatus               1.0.802    MicrosoftPowerBIMgmt.Admin               
Cmdlet          Get-PowerBIWorkspaceMigrationStatus                1.0.802    MicrosoftPowerBIMgmt.Workspaces          
Cmdlet          New-PowerBIWorkspace                               1.0.802    MicrosoftPowerBIMgmt.Workspaces          
Cmdlet          Remove-PowerBIWorkspaceUser                        1.0.802    MicrosoftPowerBIMgmt.Workspaces          
Cmdlet          Restore-PowerBIWorkspac

As you can see, there's no cmdlet for **copying** the Workspace, it's actually a separate demo script.
Use the code below to download & open it in another window.

In [3]:
(Invoke-WebRequest https://raw.githubusercontent.com/microsoft/powerbi-powershell/master/examples/CopyWorkspaceSampleScript.ps1).Content |
Out-File c:\temp\CopyWorkspaceSampleScript.ps1

### PART 3: Backup reports and datasets via Export of reports


Have a look at all the Power BI Workspaces you have access to.

In [4]:
Get-PowerBIWorkspace | Format-Table -AutoSize

If you want to simply export all of the Power BI Reports from a Workspace, you could use something along the lines of the code below:

In [ ]:
Get-PowerBIReport -WorkspaceId (Get-PowerBIWorkspace -Name 'Prod').Id |
Export-PowerBIReport

Run the PowerShell script you donwloaded and tell it which Workspace you want to copy.

In [3]:
.\CopyWorkspaceSampleScript.ps1 -SourceWorkspaceName 'Test' -TargetWorkspaceName 'Prod'
$temp_path_root = "$PSScriptRoot\pbi-copy-workspace-temp-storage"
$temp_dir = New-Item -Path $temp_path_root -ItemType Directory -ErrorAction SilentlyContinue

# STEP 3.2: Get the reports from the source workspace
$report = Get-PowerBIWorkspace | WHERE { $_.Name -eq 'BTC Testing 2' } | Get-PowerBIReport

# STEP 3.3: Export the PBIX files from the source and then import them into the target workspace
Foreach ($report in $reports) {
   
    $report_id = [guid]$report.id
    $dataset_id = [guid]$report.datasetId
    $report_name = $report.name+"_"+(Get-Date -UFormat %Y%m%d_%H%M)
    $temp_path = "$temp_path_root\$report_name.pbix"
    
    try {
        Export-PowerBIReport -Id $report_id -OutFile "$temp_path" -ErrorAction Stop
    }
    catch {
        Write-Warning "= This report and dataset cannot be copied, skipping. This is expected for most workspaces."
        continue
    }
}

After the script is finished running, check to make sure all the **Reports** are showing up in the _target_ **Workspace**.

In [6]:
Get-PowerBIReport -WorkspaceId (Get-PowerBIWorkspace -Name 'Prod').Id |
Format-Table -AutoSize